# Welcome to the API PT2!

## Data Edition

#### Goal - Filter down to the pit density we want and plot it

### Step 1. Imports

In [ ]:
# imports
from datetime import date
import geopandas as gpd
import matplotlib.pyplot as plt
from snowexsql.api import PointMeasurements, LayerMeasurements

### Step 2. Find the pits in the Boise River Basin

In [ ]:
# Find site names we can use
print(LayerMeasurements().all_site_names)

In [ ]:
# Get the first 1000 measurements from the Boise River Basin Site
df = LayerMeasurements.from_filter(
    type="density",
    site_name="Boise River Basin",
    limit=1000
)

# Explore the pits so we can find an interesting site
df.loc[:, ["site_id", "geom"]].drop_duplicates().explore()

### Step 3. Pick a point of interest

In [ ]:
# We noticed there are a lot of pits (timeseries pits) for Banner Open
# Filter down to ONE timeseries
site_id = "Banner Open"
df = LayerMeasurements.from_filter(
    type="density",
    site_id=site_id
).set_crs("epsg:26911")

df.loc[:, ["site_id", "geom"]].drop_duplicates().explore()


In [ ]:
# Get the mean of each date sampled
df["value"] = df["value"].astype(float)
df.set_index("date", inplace=True)
mean_values = df.groupby(df.index).mean()
mean_values.head()

### Notes on this mean

Taking this `mean` as bulk density **could be flawed** if layers are overlapping or layers vary in thickness

In [ ]:
# !pip install plotly
import plotly.express as px
# For rendering in readthedocs
import plotly.offline as py
py.init_notebook_mode(connected=True)

In [ ]:
# Plot the timeseries of mean density
fig = px.line(
    mean_values, x=mean_values.index, y='value',
    title=f'Mean Density - {site_id}',
    labels={'value': 'Density', 'date': 'Date'}
)

fig.update_layout(
    template='plotly_dark'
)

# Show the plot
fig.show()

# alternative matplotlib code
# mean_values["value"].plot()
# plt.title('Mean Density by Date')
# plt.xlabel('Date')
# plt.ylabel('Mean Density')
# plt.gcf().autofmt_xdate()
# plt.show()

In [ ]:
# Show more detail by using a box plot
# make sure values are floats
df["value"] = df["value"].astype(float)

# make Make a box plot
fig = px.box(df, x='date', y='value', notched=True, title='Pit Density by Date')
fig.update_traces(hoverinfo='y+name')
fig.update_layout(template='plotly_dark')

# alternative matplotlib code
# df.boxplot(by='date', column='value')
# plt.title('Density Distribution for Banner by Date')
# plt.suptitle('')  # Suppress the automatic title
# plt.xlabel('Date')
# plt.ylabel('Value')
# plt.show()